In [1]:

import torch
from torch import nn, optim



In [2]:

try:
  from utils import save_experiment, save_checkpoint
  from data import prepare_data
  from vit import ViTForClassfication
except:
  print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
  !git clone https://github.com/tintn/vision-transformer-from-scratch/
  !mv vision-transformer-from-scratch/utils.py . # get the utils.py script
  !mv vision-transformer-from-scratch/data.py . # get the data.py script
  !mv vision-transformer-from-scratch/vit.py .

[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'vision-transformer-from-scratch'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 83 (delta 43), reused 43 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 1.34 MiB | 1.45 MiB/s, done.
Resolving deltas: 100% (43/43), done.
mv: missing destination file operand after 'vision-transformer-from-scratch/vit.py'
Try 'mv --help' for more information.


In [4]:
from utils import save_experiment, save_checkpoint
from data import prepare_data
from vit import ViTForClassfication

In [5]:
# !cd vision-transformer-from-scratch/
# !ls

In [6]:

# !mv vision-transformer-from-scratch/utils.py . # get the utils.py script
# !mv vision-transformer-from-scratch/data.py . # get the data.py script
# !mv vision-transformer-from-scratch/vit.py
# !rm -rf vision-transformer-from-scratch

In [7]:


from tqdm import tqdm
config = {
    "patch_size": 4,  # Input image size: 32x32 -> 8x8 patches
    "hidden_size": 48,
    "num_hidden_layers": 4,
    "num_attention_heads": 4,
    "intermediate_size": 4 * 48, # 4 * hidden_size
    "hidden_dropout_prob": 0.0,
    "attention_probs_dropout_prob": 0.0,
    "initializer_range": 0.02,
    "image_size": 32,
    "num_classes": 10, # num_classes of CIFAR10
    "num_channels": 3,
    "qkv_bias": True,
    "use_faster_attention": True,
}
# These are not hard constraints, but are used to prevent misconfigurations
assert config["hidden_size"] % config["num_attention_heads"] == 0
assert config['intermediate_size'] == 4 * config['hidden_size']
assert config['image_size'] % config['patch_size'] == 0


class Trainer:
    """
    The simple trainer.
    """

    def __init__(self, model, optimizer, loss_fn, exp_name, device):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.exp_name = exp_name
        self.device = device

    def train(self, trainloader, testloader, epochs, save_model_every_n_epochs=0):
        """
        Train the model for the specified number of epochs.
        """
        # Keep track of the losses and accuracies
        train_losses, test_losses, accuracies = [], [], []
        # Train the model
        for i in tqdm(range(epochs), desc="Training"):
            train_loss = self.train_epoch(trainloader)
            accuracy, test_loss = self.evaluate(testloader)
            train_losses.append(train_loss)
            test_losses.append(test_loss)
            accuracies.append(accuracy)
            print(f"Epoch: {i+1}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")
            if save_model_every_n_epochs > 0 and (i+1) % save_model_every_n_epochs == 0 and i+1 != epochs:
                print('\tSave checkpoint at epoch', i+1)
                save_checkpoint(self.exp_name, self.model, i+1)
        # Save the experiment
        save_experiment(self.exp_name, config, self.model, train_losses, test_losses, accuracies)

    def train_epoch(self, trainloader):
        """
        Train the model for one epoch.
        """
        self.model.train()
        total_loss = 0
        for batch in trainloader:
            # Move the batch to the device
            batch = [t.to(self.device) for t in batch]
            images, labels = batch
            # Zero the gradients
            self.optimizer.zero_grad()
            # Calculate the loss
            loss = self.loss_fn(self.model(images)[0], labels)
            # Backpropagate the loss
            loss.backward()
            # Update the model's parameters
            self.optimizer.step()
            total_loss += loss.item() * len(images)
        return total_loss / len(trainloader.dataset)

    @torch.no_grad()
    def evaluate(self, testloader):
        self.model.eval()
        total_loss = 0
        correct = 0
        with torch.no_grad():
            for batch in testloader:
                # Move the batch to the device
                batch = [t.to(self.device) for t in batch]
                images, labels = batch

                # Get predictions
                logits, _ = self.model(images)

                # Calculate the loss
                loss = self.loss_fn(logits, labels)
                total_loss += loss.item() * len(images)

                # Calculate the accuracy
                predictions = torch.argmax(logits, dim=1)
                correct += torch.sum(predictions == labels).item()
        accuracy = correct / len(testloader.dataset)
        avg_loss = total_loss / len(testloader.dataset)
        return accuracy, avg_loss




In [8]:

def get_hyperparameters():
    hyperparameters = {
        "exp_name": "default_experiment",
        "batch_size": 256,
        "epochs": 151,
        "lr": 1e-2,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "save_model_every": 50
    }
    return hyperparameters

# Example usage:
# hyperparameters = get_hyperparameters()
# print(hyperparameters)


ViTForClassfication(
  (embedding): Embeddings(
    (patch_embeddings): PatchEmbeddings(
      (projection): Conv2d(3, 48, kernel_size=(4, 4), stride=(4, 4))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Encoder(
    (blocks): ModuleList(
      (0-3): 4 x Block(
        (attention): FasterMultiHeadAttention(
          (qkv_projection): Linear(in_features=48, out_features=144, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (output_projection): Linear(in_features=48, out_features=48, bias=True)
          (output_dropout): Dropout(p=0.0, inplace=False)
        )
        (layernorm_1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (dense_1): Linear(in_features=48, out_features=192, bias=True)
          (activation): NewGELUActivation()
          (dense_2): Linear(in_features=192, out_features=48, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (layernorm_2): LayerNorm((4

In [9]:


hyperparameters = get_hyperparameters()

# Training parameters
exp_name = hyperparameters["exp_name"]
batch_size = hyperparameters["batch_size"]
epochs = hyperparameters["epochs"]
lr = hyperparameters["lr"]
device = hyperparameters["device"]
save_model_every_n_epochs = hyperparameters["save_model_every"]
trainloader, testloader, _ = prepare_data(batch_size=batch_size)
# Create the model, optimizer, loss function and trainer
model = ViTForClassfication(config)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
loss_fn = nn.CrossEntropyLoss()
trainer = Trainer(model, optimizer, loss_fn,exp_name, device=device)
trainer.train(trainloader, testloader, epochs, save_model_every_n_epochs=save_model_every_n_epochs)


100%|██████████| 170498071/170498071 [00:12<00:00, 13489935.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Training:   0%|          | 1/500 [00:34<4:45:55, 34.38s/it]

Epoch: 1, Train loss: 1.8506, Test loss: 1.6934, Accuracy: 0.3795


Training:   0%|          | 2/500 [01:06<4:34:08, 33.03s/it]

...

Epoch: 149, Train loss: 0.5300, Test loss: 0.6877, Accuracy: 0.7700


Training:  30%|███       | 150/500 [1:23:27<3:17:46, 33.90s/it]

Epoch: 150, Train loss: 0.5218, Test loss: 0.6888, Accuracy: 0.7687


Training:  30%|███       | 150/500 [1:23:30<3:14:51, 33.40s/it]


KeyboardInterrupt: 

In [18]:

import torch

def save_checkpoint(model, optimizer, epoch, path="checkpoint.pth", scheduler=None):
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    }
    if scheduler is not None:
        checkpoint["scheduler_state_dict"] = scheduler.state_dict()

    torch.save(checkpoint, path)
    print(f"Checkpoint saved at epoch {epoch}.")

# Example usage:

model = trainer.model
optimizer = trainer.optimizer
scheduler = None
epoch =  150
save_checkpoint(model, optimizer, epoch, path="checkpoint_1.pth", scheduler=scheduler)




Checkpoint saved at epoch 150.


In [16]:


def load_checkpoint(path="checkpoint.pth", model=None, optimizer=None, scheduler=None):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    if scheduler is not None:
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])

    epoch = checkpoint["epoch"]
    print(f"Checkpoint loaded. Resuming from epoch {epoch}.")

    return epoch

# Example usage:
# start_epoch = load_checkpoint(path="my_checkpoint.pth", model=model, optimizer=optimizer, scheduler=scheduler)



In [10]:
raise EOFError

NameError: name 'model' is not defined

In [11]:

epochs

NameError: name 'epochs' is not defined

In [12]:

optimizer

NameError: name 'optimizer' is not defined

In [ ]:
def load_checkpoint(path="checkpoint.pth", model=None, optimizer=None, scheduler=None):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    if scheduler is not None:
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])

    epoch = checkpoint["epoch"]
    print(f"Checkpoint loaded. Resuming from epoch {epoch}.")

    return epoch

# Example usage:
start_epoch = load_checkpoint(path="my_checkpoint.pth", model=model, optimizer=optimizer, scheduler=scheduler)


In [ ]:

raise ZeroDivisionError